In [ ]:
# default_exp uniformcf

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
# stellt sicher, dass beim verändern der core library diese wieder neu geladen wird
%load_ext autoreload
%autoreload 2

# Uniform CF

## Basic Settings

In [ ]:
# imports
from bfh_mt_hs2020_sec_data.core import * 
from pathlib import Path
from typing import List, Tuple, Union, Set
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col

import pandas as pd

import shutil          # provides high level file operations
import time            # used to measure execution time
import os
import sys

In [ ]:
all_pivot_selected_folder  = "D:/data_mt/03_1_selected_pivot_cols_parquet" 
all_pivoted_folder         = "D:/data_mt/03_2_split_pivot_parquet"         
all_processed_folder       = "D:/data_mt/04_unified/"

col_list =    ["stmt","cik","ticker", "adsh","period","filed", "form","tag","value","report", "line", "fp", "uom"]
pivot_group = ["cik","ticker","adsh","form","period","filed","fp", "qtrs"]

In [ ]:
# init Spark
spark = get_spark_session() # Session anlegen
spark # display the moste important information of the session

## 00_Raw_data

In [ ]:
# loading the complete unpivoted dataset - if it is needed for debbuging
df_all_selected = spark.read.parquet(all_pivot_selected_folder).cache()

In [ ]:
# it happens sometimes, that the data could not be associated with a right sheet (bs, is, cf, ..). in this cases, the data can appea under "UN"
# so if expected information cannot be found in the appropriate statement, we have to look in the un statement
un_pivot_value = load_data(all_pivoted_folder, spark, "UN", "value")
un_pivot_value = un_pivot_value.where("((form == '10-K' and qtrs == '4') or (form == '10-Q' and ((fp  == 'Q1' and qtrs == '1') or (fp  == 'Q2' and qtrs == '2') or (fp  == 'Q3' and qtrs == '3') or (fp  == 'Q4' and qtrs == '4'))))")
un_pivot_pd = un_pivot_value.toPandas()

In [ ]:
un_pivot_pd.shape

(2428, 1775)

In [ ]:
def prepare_un_values(df_to_merge_into, attr_list):
    # add possible columns from un set to  cf data with prefix cpy_
    attributes = pivot_group[:] # create copy
    attributes.extend(attr_list)

    un_prepared = un_pivot_pd[attributes].copy()
    un_prepared.rename(columns=lambda x: x  if x in pivot_group else ("cpy_" + x), inplace=True)

    return pd.merge(df_to_merge_into, un_prepared, how='left', on=pivot_group)

## 01_CashFlow

Operation
- NetIncomeLoss
- ProfitLoss
- NetCashProvidedByUsedInOperatingActivities: NetIncome + other positions ergibt diese Position

Investing
- NetCashProvidedByUsedInInvestingActivities

Financing activities
- PaymentsForRepurchaseOfCommonStock: Aktienrückkäufe
- PaymentsOfDividends
- NetCashProvidedByUsedInFinancingActivities

Cash Bestand unterschied
- CashAndCashEquivalentsPeriodIncreaseDecrease: increase/decrease in cash

In [ ]:
cf_pivot_value = load_data(all_pivoted_folder, spark, "CF", "value")
spark_shape(cf_pivot_value)

(116187, 2436)

In [ ]:
cf_pivot_pd = cf_pivot_value.toPandas()

In [ ]:
cf_pivot_pd_copy = cf_pivot_pd.copy()

In [ ]:
cf_pivot_pd_copy.shape

(116187, 2436)

### Merge Data from "unknown" statement

In [ ]:
# merge relevant columns from the UN dataset
cf_pivot_pd_copy = prepare_un_values(cf_pivot_pd_copy, [
    'CashAndCashEquivalentsPeriodIncreaseDecrease',
     # 'CashAndCashEquivalentsPeriodIncreaseDecreaseExcludingExchangeRateEffect',
    'CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseExcludingExchangeRateEffect',
    'CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect',   
    # 'CashPeriodIncreaseDecrease',
    # 'CashPeriodIncreaseDecreaseExcludingExchangeRateEffect',
    'NetCashProvidedByUsedInContinuingOperations',
    
    'ProfitLoss',
    'NetIncomeLoss', 
    'NetIncomeLossAvailableToCommonStockholdersBasic',
    'NetCashProvidedByUsedInOperatingActivities',
    'NetCashProvidedByUsedInOperatingActivitiesContinuingOperations',
 
    'NetCashProvidedByUsedInInvestingActivities',
    'NetCashProvidedByUsedInFinancingActivities',
    'NetCashProvidedByUsedInInvestingActivitiesContinuingOperations',
    'CashProvidedByUsedInInvestingActivitiesDiscontinuedOperations',
    
    'NetCashProvidedByUsedInFinancingActivitiesContinuingOperations',
    # 'CashProvidedByUsedInFinancingActivitiesDiscontinuedOperations' ,
])
cf_pivot_pd_copy.shape

(116187, 2450)

### Cash Increase/Decrease
- 'CashAndCashEquivalentsPeriodIncreaseDecrease',
- 'CashAndCashEquivalentsPeriodIncreaseDecreaseExcludingExchangeRateEffect',
- 'CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseExcludingExchangeRateEffect',
- 'CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect',
- 'CashPeriodIncreaseDecrease',
- 'CashPeriodIncreaseDecreaseExcludingExchangeRateEffect',
- 'NetCashProvidedByUsedInContinuingOperations'

In [ ]:
print_null_count(cf_pivot_pd_copy, ['CashAndCashEquivalentsPeriodIncreaseDecrease'])

CashAndCashEquivalentsPeriodIncreaseDecrease   29201


In [ ]:
cf_pivot_pd_copy['c_CashIncreaseDecrease'] = None

copy_if_not_empty(cf_pivot_pd_copy, 'cpy_CashAndCashEquivalentsPeriodIncreaseDecrease', 'CashAndCashEquivalentsPeriodIncreaseDecrease')
copy_if_not_empty(cf_pivot_pd_copy, 'cpy_CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseExcludingExchangeRateEffect', 'CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseExcludingExchangeRateEffect')
copy_if_not_empty(cf_pivot_pd_copy, 'cpy_CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect', 'CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect')


copy_if_not_empty(cf_pivot_pd_copy, 'CashAndCashEquivalentsPeriodIncreaseDecrease', 'c_CashIncreaseDecrease')
copy_if_not_empty(cf_pivot_pd_copy, 'CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect', 'c_CashIncreaseDecrease') # either or
copy_if_not_empty(cf_pivot_pd_copy, 'CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseExcludingExchangeRateEffect', 'c_CashIncreaseDecrease') # either or
copy_if_not_empty(cf_pivot_pd_copy, 'CashPeriodIncreaseDecrease', 'c_CashIncreaseDecrease') # either or
copy_if_not_empty(cf_pivot_pd_copy, 'CashAndCashEquivalentsPeriodIncreaseDecreaseExcludingExchangeRateEffect', 'c_CashIncreaseDecrease') # either or
copy_if_not_empty(cf_pivot_pd_copy, 'CashPeriodIncreaseDecreaseExcludingExchangeRateEffect', 'c_CashIncreaseDecrease') # either or

In [ ]:
print_null_count(cf_pivot_pd_copy, ['c_CashIncreaseDecrease'])

c_CashIncreaseDecrease   8566


### NetIncomeLoss / Operation CashFlow

In [ ]:
print_null_count(cf_pivot_pd_copy, ['NetCashProvidedByUsedInOperatingActivities'])

NetCashProvidedByUsedInOperatingActivities   30576


In [ ]:
cf_pivot_pd_copy['c_CashFromOperating'] = None

copy_if_not_empty(cf_pivot_pd_copy, 'cpy_NetCashProvidedByUsedInOperatingActivities', 'NetCashProvidedByUsedInOperatingActivities')
copy_if_not_empty(cf_pivot_pd_copy, 'cpy_NetCashProvidedByUsedInOperatingActivitiesContinuingOperations', 'NetCashProvidedByUsedInOperatingActivitiesContinuingOperations')

copy_if_not_empty(cf_pivot_pd_copy, 'NetCashProvidedByUsedInOperatingActivities', 'c_CashFromOperating')
copy_if_not_empty(cf_pivot_pd_copy, 'NetCashProvidedByUsedInOperatingActivitiesContinuingOperations', 'c_CashFromOperating')

In [ ]:
print_null_count(cf_pivot_pd_copy, ['c_CashFromOperating'])

c_CashFromOperating   377


### Investing
- NetCashProvidedByUsedInInvestingActivities

In [ ]:
print_null_count(cf_pivot_pd_copy, ['NetCashProvidedByUsedInInvestingActivities'])

NetCashProvidedByUsedInInvestingActivities   31288


In [ ]:
cf_pivot_pd_copy['c_CashFromInvesting'] = None

copy_if_not_empty(cf_pivot_pd_copy, 'cpy_NetCashProvidedByUsedInInvestingActivities', 'NetCashProvidedByUsedInInvestingActivities')
copy_if_not_empty(cf_pivot_pd_copy, 'cpy_NetCashProvidedByUsedInInvestingActivitiesContinuingOperations','NetCashProvidedByUsedInInvestingActivitiesContinuingOperations')
copy_if_not_empty(cf_pivot_pd_copy, 'cpy_CashProvidedByUsedInInvestingActivitiesDiscontinuedOperations' ,'CashProvidedByUsedInInvestingActivitiesDiscontinuedOperations')

copy_if_not_empty(cf_pivot_pd_copy, 'NetCashProvidedByUsedInInvestingActivities', 'c_CashFromInvesting')
copy_if_not_empty(cf_pivot_pd_copy, 'NetCashProvidedByUsedInInvestingActivitiesContinuingOperations', 'c_CashFromInvesting')
copy_if_not_empty(cf_pivot_pd_copy, 'CashProvidedByUsedInInvestingActivitiesDiscontinuedOperations', 'c_CashFromInvesting')

set_to_zero_if_null(cf_pivot_pd_copy, 'c_CashFromInvesting') # it is absolutely possible, that there are no investing activities

In [ ]:
print_null_count(cf_pivot_pd_copy, ['c_CashFromInvesting'])

c_CashFromInvesting   0


### Financing activities

#### NetCashProvidedByUsedInFinancingActivities

In [ ]:
print_null_count(cf_pivot_pd_copy, ['NetCashProvidedByUsedInFinancingActivities'])

NetCashProvidedByUsedInFinancingActivities   31409


In [ ]:
cf_pivot_pd_copy['c_CashFromFinancing'] = None

copy_if_not_empty(cf_pivot_pd_copy, 'cpy_NetCashProvidedByUsedInFinancingActivities', 'NetCashProvidedByUsedInFinancingActivities')
copy_if_not_empty(cf_pivot_pd_copy, 'cpy_NetCashProvidedByUsedInFinancingActivitiesContinuingOperations','NetCashProvidedByUsedInFinancingActivitiesContinuingOperations')

sum_into_empty_target(cf_pivot_pd_copy, 
                      'NetCashProvidedByUsedInFinancingActivitiesContinuingOperations',
                      'CashProvidedByUsedInFinancingActivitiesDiscontinuedOperations' ,
                      'NetCashProvidedByUsedInFinancingActivities')

copy_if_not_empty(cf_pivot_pd_copy, 'NetCashProvidedByUsedInFinancingActivities', 'c_CashFromFinancing')
copy_if_not_empty(cf_pivot_pd_copy, 'NetCashProvidedByUsedInFinancingActivitiesContinuingOperations', 'c_CashFromFinancing')
copy_if_not_empty(cf_pivot_pd_copy, 'CashProvidedByUsedInFinancingActivitiesDiscontinuedOperations', 'c_CashFromFinancing')

set_to_zero_if_null(cf_pivot_pd_copy, 'c_CashFromFinancing') # it is absolutely possible, that there are no financing activities

In [ ]:
print_null_count(cf_pivot_pd_copy, ['c_CashFromFinancing'])

c_CashFromFinancing   0


#### PaymentsOfDividends
Simply set to 0.0 if no data is present
 'PaymentsOfDividends',
 'PaymentsOfDividendsCommonStock',
 'PaymentsOfDividendsMinorityInterest',
 'PaymentsOfDividendsPreferredStockAndPreferenceStock',
 'PaymentsOfOrdinaryDividends',

In [ ]:
cf_pivot_pd_copy = sum_cols_into_new_target(cf_pivot_pd_copy, 'c_PaymentsOfDividendsTotal', 
                                 ['PaymentsOfDividends',
                                 'PaymentsOfDividendsCommonStock',
                                 'PaymentsOfDividendsMinorityInterest',
                                 'PaymentsOfDividendsPreferredStockAndPreferenceStock',
                                 'PaymentsOfOrdinaryDividends'])

#### PaymentsForRepurchaseOfCommonStock: Stock buybacks
 'PaymentsForRepurchaseOfCommonStock',
 'PaymentsForRepurchaseOfCommonStockForEmployeeTaxWithholdingObligations',
 'PaymentsForRepurchaseOfConvertiblePreferredStock',
 'PaymentsForRepurchaseOfPreferredStockAndPreferenceStock',
 'PaymentsForRepurchaseOfRedeemableConvertiblePreferredStock',
 'PaymentsForRepurchaseOfRedeemablePreferredStock'

In [ ]:
cf_pivot_pd_copy = sum_cols_into_new_target(cf_pivot_pd_copy, 'c_PaymentsForRepurchaseOfStockTotal', 
                                 ['PaymentsForRepurchaseOfCommonStock',
                                     'PaymentsForRepurchaseOfCommonStockForEmployeeTaxWithholdingObligations',
                                     'PaymentsForRepurchaseOfConvertiblePreferredStock',
                                     'PaymentsForRepurchaseOfPreferredStockAndPreferenceStock',
                                     'PaymentsForRepurchaseOfRedeemableConvertiblePreferredStock',
                                     'PaymentsForRepurchaseOfRedeemablePreferredStock'])

### Corrections
some Bookings are simply wrong. There are a few cases, when PayedDividends Appear as an inflow, which is wrong. The same holds true for PaymentsForRepurchaseOfStockTotal

In [ ]:
cf_pivot_pd_copy.loc[cf_pivot_pd_copy.c_PaymentsOfDividendsTotal>0,'c_PaymentsOfDividendsTotal'] = -cf_pivot_pd_copy.loc[cf_pivot_pd_copy.c_PaymentsOfDividendsTotal>0].c_PaymentsOfDividendsTotal
cf_pivot_pd_copy.loc[cf_pivot_pd_copy.c_PaymentsForRepurchaseOfStockTotal>0,'c_PaymentsForRepurchaseOfStockTotal'] = -cf_pivot_pd_copy.loc[cf_pivot_pd_copy.c_PaymentsForRepurchaseOfStockTotal>0].c_PaymentsForRepurchaseOfStockTotal

### Save

In [ ]:
cf_pivot_pd_copy[["cik","ticker", "adsh","period","filed", "form", "qtrs","fp",
                                     'c_CashIncreaseDecrease',
                                     'c_CashFromOperating', 
                                     'c_CashFromInvesting',
                                     'c_CashFromFinancing',
                                     'c_PaymentsOfDividendsTotal',
                                     'c_PaymentsForRepurchaseOfStockTotal']] \
                  .to_csv(all_processed_folder + "04_cf_unified.csv", index=False)

In [ ]:
spark.stop()

## XX_Trials

In [ ]:
div_data = cf_pivot_pd_copy[cf_pivot_pd_copy.c_PaymentsForRepurchaseOfStockTotal>0][['ticker','cik','adsh', 'c_PaymentsForRepurchaseOfStockTotal',
                                     'PaymentsForRepurchaseOfCommonStock',
                                     'PaymentsForRepurchaseOfCommonStockForEmployeeTaxWithholdingObligations',
                                     'PaymentsForRepurchaseOfConvertiblePreferredStock',
                                     'PaymentsForRepurchaseOfPreferredStockAndPreferenceStock',
                                     'PaymentsForRepurchaseOfRedeemableConvertiblePreferredStock',
                                     'PaymentsForRepurchaseOfRedeemablePreferredStock']]

In [ ]:
print(cf_pivot_pd_copy[cf_pivot_pd_copy.c_PaymentsForRepurchaseOfStockTotal>0].shape[0])
print(div_data.shape[0])
print(len(div_data.ticker.unique()))

547
547
218


In [ ]:
div_data[div_data.ticker=='AAPL'].sort_values('adsh')

,ticker,cik,adsh,c_PaymentsForRepurchaseOfStockTotal,PaymentsForRepurchaseOfCommonStock,PaymentsForRepurchaseOfCommonStockForEmployeeTaxWithholdingObligations,PaymentsForRepurchaseOfConvertiblePreferredStock,PaymentsForRepurchaseOfPreferredStockAndPreferenceStock,PaymentsForRepurchaseOfRedeemableConvertiblePreferredStock,PaymentsForRepurchaseOfRedeemablePreferredStock


In [ ]:
aapl_pd = df_all_selected.where("adsh == '0000320193-17-000070' and stmt = 'CF'").toPandas()

In [ ]:
aapl_pd

In [ ]:
# cf_pivot_value.select('adsh').distinct().count() # 116482
# cf_pivot_value.select('adsh','form').where('form == "10-K"').distinct().count() # 27556
# cf_pivot_value.select('adsh','form').where('form == "10-Q"').distinct().count() # 88926

# cf_pivot_value.select('adsh','form', 'qtrs').where('form == "10-K" and qtrs="0"').distinct().count() # 27264
# cf_pivot_value.select('adsh','form', 'qtrs').where('form == "10-K" and qtrs="1"').distinct().count() # 16405
# cf_pivot_value.select('adsh','form', 'qtrs').where('form == "10-K" and qtrs="2"').distinct().count() # 63
# cf_pivot_value.select('adsh','form', 'qtrs').where('form == "10-K" and qtrs="3"').distinct().count() # 44
cf_pivot_value.select('adsh','form', 'qtrs').where('form == "10-K" and qtrs="4"').distinct().count() # 27497

# cf_pivot_value.select('adsh','form', 'qtrs').where('form == "10-Q" and qtrs="0"').distinct().count() # 88076
# cf_pivot_value.select('adsh','form', 'qtrs').where('form == "10-Q" and qtrs="1"').distinct().count() # 86728
# cf_pivot_value.select('adsh','form', 'qtrs').where('form == "10-Q" and qtrs="2"').distinct().count() # 30215
# cf_pivot_value.select('adsh','form', 'qtrs').where('form == "10-Q" and qtrs="3"').distinct().count() # 28150
# cf_pivot_value.select('adsh','form', 'qtrs').where('form == "10-Q" and qtrs="4"').distinct().count() # 185

In [ ]:
piv_by_qtrs_pd = cf_pivot_value.select('adsh','fp','form','qtrs').groupby(['adsh','fp','form']).pivot('qtrs').count().toPandas()


#pd_frame = df_all_selected.where("adsh == '0001564590-20-043606' and stmt == 'IS' and qtrs=='1'").toPandas()
#pd_frame = df_all_selected.where("adsh == '0001628279-20-000210'").toPandas()
#pd_frame = df_all_selected.where("adsh == '0001193125-20-213555'").toPandas()
#print(pd_frame.sort_values(['report', 'line']))

In [ ]:
pd.set_option('display.max_rows', 20)
piv_by_qtrs_pd[['adsh','fp','form','0','1','2','3','4']]

,adsh,fp,form,0,1,2,3,4
0,0001193125-14-434590,Q2,10-Q,1.0,1.0,1.0,NaN,NaN
1,0001283699-17-000032,Q2,10-Q,1.0,1.0,1.0,NaN,NaN
2,0000946673-19-000012,Q1,10-Q,1.0,1.0,NaN,NaN,NaN
3,0001558370-19-008195,Q2,10-Q,1.0,1.0,1.0,NaN,NaN
4,0001654954-19-002486,FY,10-K,1.0,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...
116477,0001564590-17-007757,Q1,10-Q,1.0,1.0,NaN,NaN,NaN
116478,0001437749-12-004522,Q1,10-Q,1.0,1.0,NaN,NaN,NaN
116479,0000078890-17-000026,Q1,10-Q,1.0,1.0,NaN,NaN,NaN
116480,0001193125-14-284514,Q2,10-Q,1.0,1.0,1.0,NaN,NaN


In [ ]:
# 0001392972-17-000128  Q3 / qtrs 0,1,3 -> 3 voll
pd_frame = cf_pivot_value.where("adsh == '0001193125-13-315367'").toPandas()

In [ ]:
pd_frame = cf_pivot_value.where("adsh == ' 0001392972-17-000128'").toPandas()
pd.set_option('display.max_rows', pd_frame.shape[0]+1)
pd_frame.sort_values(['qtrs']).dropna(how='all', axis=1)

,cik,ticker,adsh,form,period,filed,fp,qtrs,AmortizationOfFinancingCostsAndDiscounts,CashAndCashEquivalentsAtCarryingValue,...,PaymentsToAcquireIntangibleAssets,PaymentsToAcquirePropertyPlantAndEquipment,PaymentsToAcquireShortTermInvestments,ProceedsFromConvertibleDebt,ProceedsFromMaturitiesPrepaymentsAndCallsOfAvailableForSaleSecurities,ProceedsFromStockOptionsExercised,ProceedsFromStockPlans,ProvisionForDoubtfulAccounts,RepaymentsOfNotesPayable,ShareBasedCompensation
1,1392972,PRO,0001392972-17-000128,10-Q,2017-09-30,2017-10-26,Q3,0,NaN,157359000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1392972,PRO,0001392972-17-000128,10-Q,2017-09-30,2017-10-26,Q3,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1392972,PRO,0001392972-17-000128,10-Q,2017-09-30,2017-10-26,Q3,3,6363000.0,NaN,...,75000.0,1235000.0,0.0,93500000.0,15992000.0,6347000.0,1535000.0,0.0,155000.0,17665000.0


In [ ]:
pd_frame = cf_pivot_value.where("adsh == '0001193125-13-315367'").toPandas()
pd.set_option('display.max_rows', pd_frame.shape[0]+1)
pd_frame.sort_values(['qtrs']).dropna(how='all', axis=1)

,cik,ticker,adsh,form,period,filed,fp,qtrs,CashAndCashEquivalentsAtCarryingValue,CashAndCashEquivalentsPeriodIncreaseDecrease,...,PaymentsForRepurchaseOfCommonStock,PaymentsToAcquireOtherInvestments,PaymentsToAcquirePropertyPlantAndEquipment,PaymentsToAcquireTradingSecuritiesHeldforinvestment,ProceedsFromSaleAndMaturityOfAvailableForSaleSecurities,ProceedsFromSaleAndMaturityOfOtherInvestments,ProceedsFromSaleAndMaturityOfTradingSecuritiesHeldforinvestment,ProceedsFromStockOptionsExercised,ShareBasedCompensation,UnrealizedGainLossOnInvestments
2,1163348,AWH,0001193125-13-315367,10-Q,2013-06-30,2013-08-02,Q2,0,674103000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1163348,AWH,0001193125-13-315367,10-Q,2013-06-30,2013-08-02,Q2,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1163348,AWH,0001193125-13-315367,10-Q,2013-06-30,2013-08-02,Q2,2,NaN,-7776000.0,...,82571000.0,141805000.0,3363000.0,3.186162e+09,0.0,126491000.0,3.171977e+09,5293000.0,6566000.0,-95698000.0


In [ ]:
pd_frame = cf_pivot_value.where("adsh == '0000946673-19-000012'").toPandas()
pd.set_option('display.max_rows', pd_frame.shape[0]+1)
pd_frame.sort_values(['qtrs']).dropna(how='all', axis=1)

,cik,ticker,adsh,form,period,filed,fp,qtrs,CashAndCashEquivalentsAtCarryingValue,CashAndCashEquivalentsPeriodIncreaseDecrease,...,ProceedsFromMaturitiesPrepaymentsAndCallsOfHeldToMaturitySecurities,ProceedsFromSaleOfAvailableForSaleSecurities,ProceedsFromSaleOfFederalHomeLoanBankStock,ProceedsFromSaleOfLoansHeldForInvestment,ProceedsFromSaleOfLoansHeldForSale,ProceedsFromSaleOfPropertyPlantAndEquipment,ProvisionForLoanAndLeaseLosses,ProvisionForLoanLeaseAndOtherLosses,ShareBasedCompensation,TransferToOtherRealEstate
0,946673,BANR,0000946673-19-000012,10-Q,2019-03-31,2019-05-03,Q1,0,261538000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,946673,BANR,0000946673-19-000012,10-Q,2019-03-31,2019-05-03,Q1,1,NaN,-10658000.0,...,14744000.0,516000.0,52372000.0,3186000.0,261978000.0,876000.0,2000000.0,0.0,1219000.0,0.0


In [ ]:
df_all_selected.where("adsh == '0001654954-19-002486' and stmt = 'CF'").orderBy('line').show(50,False)

+----+-------+------+--------------------+----------+----+----------+------------------------------------------------------+----------+------+----+---+---+----+-----------+
|stmt|cik    |ticker|adsh                |period    |form|filed     |tag                                                   |value     |report|line|fp |uom|qtrs|period_year|
+----+-------+------+--------------------+----------+----+----------+------------------------------------------------------+----------+------+----+---+---+----+-----------+
|CF  |1104855|SPRT  |0001654954-19-002486|2018-12-31|10-K|2019-03-08|NetIncomeLoss                                         |-9100000.0|7     |2   |FY |USD|4   |2018       |
|CF  |1104855|SPRT  |0001654954-19-002486|2018-12-31|10-K|2019-03-08|ShareBasedCompensation                                |680000.0  |7     |4   |FY |USD|4   |2018       |
|CF  |1104855|SPRT  |0001654954-19-002486|2018-12-31|10-K|2019-03-08|AmortizationOfIntangibleAssets                        |0.0       |